Mississippi Stud Simulation Jupyter Notebook

In [9]:
import os
os.chdir("C:\\Users\\giria\\OneDrive\\Desktop\\Desktop\\PDFs and Text Docs\\Coding Stuff\\Casino411\\jacobson_m_stud\\msstud_trainer")

# Imports
import multiprocessing
import matplotlib.pyplot as plt
import numpy as np
from card_lib.deck import Deck
from card_lib.simulation.mississippi_simulator import simulate_round
from card_lib.simulation.mississippi_simulator import MississippiStudStrategy
from core.strategies.basic import BasicStrategy
from core.strategies.ap3 import AdvantagePlay3rdStrategy
from core.strategies.ap5 import AdvantagePlay5thStrategy

In [10]:
# Simulation Parameters
ROUNDS = 10000
ANTE = 5
STRATEGY = "ap3"  # options: "basic", "ap3", "ap5"

In [11]:
# Strategy Selection
if STRATEGY == "basic":
    strategy = BasicStrategy()
elif STRATEGY == "ap3":
    strategy = AdvantagePlay3rdStrategy()
elif STRATEGY == "ap5":
    strategy = AdvantagePlay5thStrategy()
else:
    raise ValueError("Unknown strategy")

In [12]:
class NotebookStrategyWrapper(MississippiStudStrategy):
    def __init__(self, strategy):
        self.strategy = strategy

    def get_bet(self, hole_cards, revealed_community_cards, stage, ante=1, current_total=0, peeked_cards=None):
        return self.strategy.get_bet(hole_cards, revealed_community_cards, stage, ante, current_total, peeked_cards)

In [13]:
def run_worker(strategy_class_name, ante):
    deck = Deck()
    deck.shuffle()
    strategy_cls = BasicStrategy if strategy_class_name == "basic" else (AdvantagePlay3rdStrategy if strategy_class_name == "ap3" else AdvantagePlay5thStrategy)
    strategy = strategy_cls()
    return simulate_round(deck, strategy, ante, ap_revealed_community_cards={'3rd': True if strategy_class_name == "ap3" else False, '4th': False, '5th': True if strategy_class_name == "ap5" else False})

In [14]:
def run_parallel_sim(strategy_name="basic", rounds=10000, ante=5, workers=None):
    with multiprocessing.Pool(processes=workers or multiprocessing.cpu_count()) as pool:
        args = [(strategy_name, ante)] * rounds
        profits = pool.starmap(run_worker, args)

    profits = np.array(profits)
    ev = np.mean(profits)
    std_dev = np.std(profits)
    win_rate = np.mean(profits > 0)
    loss_rate = np.mean(profits < 0)
    push_rate = np.mean(profits == 0)

    print(f"EV/hand: ${ev:.2f}")
    print(f"Std Dev: ${std_dev:.2f}")
    print(f"Win: {win_rate*100:.1f}%, Loss: {loss_rate*100:.1f}%, Push: {push_rate*100:.1f}%")

    plt.hist(profits, bins=50, alpha=0.7, edgecolor='black')
    plt.title("Profit Distribution")
    plt.xlabel("Profit")
    plt.ylabel("Frequency")
    plt.grid(True)
    plt.show()

In [1]:
run_parallel_sim(strategy_name="ap3", rounds=1, ante=5, workers=4)

NameError: name 'run_parallel_sim' is not defined